#### Building Model

In [1]:
# -*- coding: utf-8 -*-
'''VGG16-places365 model for Keras
# Reference:
- [Places: A 10 million Image Database for Scene Recognition](http://places2.csail.mit.edu/PAMI_places.pdf)
'''

from __future__ import division, print_function
import os

import warnings
import numpy as np

from keras import backend as K
from keras.layers import Input
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras.layers import Conv2D
from keras.regularizers import l2
from keras.layers.core import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils import layer_utils
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16_Places365(include_top=True, weights='places',
                    input_tensor=None, input_shape=None,
                    pooling=None,
                    classes=365):
    """Instantiates the VGG16-places365 architecture.
    Optionally loads weights pre-trained
    on Places. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization),
                 'places' (pre-training on Places),
                 or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`, or invalid input shape
        """
    if not (weights in {'places', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `places` '
                         '(pre-training on Places), '
                         'or the path to the weights file to be loaded.')

    if weights == 'places' and include_top and classes != 365:
        raise ValueError('If using `weights` as places with `include_top`'
                         ' as true, `classes` should be 365')


    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten =include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    # Block 1
    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block1_conv1')(img_input)

    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block1_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool", padding='valid')(x)

    # Block 2
    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block2_conv1')(x)

    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block2_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool", padding='valid')(x)

    # Block 3
    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv1')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv2')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool", padding='valid')(x)

    # Block 4
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool", padding='valid')(x)

    # Block 5
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool", padding='valid')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dropout(0.5, name='drop_fc1')(x)

        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dropout(0.5, name='drop_fc2')(x)
        
        x = Dense(365, activation='softmax', name="predictions")(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create model.
    model = Model(inputs, x, name='vgg16-places365')

    # load weights
    if weights == 'places':
        if include_top:
            weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')

        model.load_weights(weights_path)

        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')

    elif weights is not None:
        model.load_weights(weights)

    return model



Using TensorFlow backend.


#### Create Model

In [2]:
from keras import optimizers
def create_model():
    
    model = VGG16_Places365(include_top = False, weights='places', pooling = "avg")
    
    # Freeze all layers except the last 4 layers
    for layer in model.layers[:-4]:
        layer.trainable = False

    x = model.output
#     x = Dropout(0.4)(x)
    predictions = Dense(6, activation='sigmoid')(x)
    model = Model(inputs=model.input, outputs=predictions)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=0.001))
    
    return model

#### Load Weights with the Lowest Validation Loss

In [3]:
model = create_model()
model.load_weights("weights-improvement-112-0.90.hdf5")
print(model.summary())


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________

#### check if there's any corrupted file

In [10]:
from tqdm import tqdm
from PIL import Image
DATADIR = r"D:\FYP\KL_GSV"

# CATEGORIES = ["beautiful","boring","depressing","lively","safety","wealthy"]

training_data = []

def checkCorrupted():
#     for category in CATEGORIES:  
    path = DATADIR  # create path to dogs and cats
#         class_num = CATEGORIES.index(category)  # get the classification  (0 == beautiful, ... ,5 == wealthy)
    for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
#         try:
        filename = os.path.join(path,img)
#             print(filename)
        im = Image.open(filename)
        size = os.path.getsize(filename)
        if (size == 3224) :
            print(filename)

checkCorrupted()

100%|████████████████████████████████████████████████████████████████████████████| 4422/4422 [00:00<00:00, 6124.07it/s]


#### Delete record from the DF if corrupted file in the directory is deleted

In [60]:
dir = r"D:\FYP\KL_GSV"
print(len(df))
imagePath = df.loc[:,'filename'].values.tolist()

count = 0
for img in imagePath:
    path = os.path.join(dir, img)
    x = str(path).split("\\")[-1]
    if not os.path.exists(path):
        count += 1
        print(x)
        df = df[df.filename != x]
#         print(len(df))
print(count)
print(len(df))
df.to_csv(r"D:\FYP\Visualisation\test_points_KL_unique_cleaned_latest.csv")

4291
0
4291


#### Prediction

In [62]:
import cv2
import numpy as np
import os
from sklearn.metrics import classification_report, average_precision_score, precision_score
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# classes = ["beautiful","boring","depressing","lively","safety","wealthy"]

predicted_scores = []

dir = r"D:\FYP\KL_GSV"
# imagePath = df.loc[df['Unnamed: 0'] > 20052]
imagePath = df.loc[:,'filename'].values.tolist()
print(len(imagePath))


for img in imagePath:
    try:
        path = os.path.join(dir, img)

        idx = imagePath.index(img)
        print(idx)
    #     print(imagePath.index(img))

        img = cv2.imread(path)

        y_scores = model.predict(img.reshape(1,224,224,3))
        print(path)
        print(y_scores[0])
        predicted_scores.append(y_scores[0])
    except:
        print("deleted file")

4291
0
D:\FYP\KL_GSV\ChIJYbEOt2Y4zDER1rRiQCmnKpk_0.jpg
[0.958 0.087 0.274 0.950 0.480 0.792]
1
D:\FYP\KL_GSV\ChIJf7L63GY4zDERtn4RRBJ5j8g_0.jpg
[0.824 0.093 0.899 0.969 0.008 0.261]
2
D:\FYP\KL_GSV\ChIJWZXLyGY4zDER8HrwgxBiXIM_0.jpg
[0.993 0.005 0.006 0.996 0.002 0.996]
3
D:\FYP\KL_GSV\ChIJBRqW62Y4zDER5hk-VslDi6o_0.jpg
[0.961 0.643 0.784 0.998 0.213 0.614]
4
D:\FYP\KL_GSV\ChIJVVCnx2Y4zDERuMiq6zyzIXs_0.jpg
[0.993 0.525 0.952 0.972 0.009 0.992]
5
D:\FYP\KL_GSV\ChIJ4XieRGE4zDERdtE5f_yE81g_0.jpg
[0.301 0.938 0.643 0.495 0.005 0.991]
6
D:\FYP\KL_GSV\ChIJh7QNS2E4zDER8uQZQewf8Vo_0.jpg
[0.923 0.155 0.051 0.997 0.006 0.999]
7
D:\FYP\KL_GSV\ChIJB6-MrWY4zDERQE4s69HmJfM_0.jpg
[0.649 0.735 0.949 1.000 0.221 0.954]
8
D:\FYP\KL_GSV\ChIJQ1hPOGE4zDERdkZjREI-6no_0.jpg
[0.691 0.950 0.014 0.999 0.002 0.998]
9
D:\FYP\KL_GSV\ChIJbQmI-WM4zDERBWb7oyT8gVs_0.jpg
[0.994 0.958 0.065 0.819 0.005 0.935]
10
D:\FYP\KL_GSV\ChIJI1LPF2E4zDERuiUvyloEyB4_0.jpg
[0.812 0.860 0.853 0.980 0.539 0.959]
11
D:\FYP\KL_GSV\ChIJ5Wy9W

D:\FYP\KL_GSV\ChIJC3eqyhI4zDER72zVEB2S35M_9.jpg
[0.636 0.997 0.998 0.982 0.000 0.424]
184
D:\FYP\KL_GSV\ChIJ_dGZlRI4zDERWM5nZSd05k0_9.jpg
[0.984 0.993 0.045 0.999 0.002 0.997]
185
D:\FYP\KL_GSV\ChIJC3eqyhI4zDER7mzVEB2S35M_9.jpg
[0.673 0.996 0.993 0.971 0.000 0.194]
186
D:\FYP\KL_GSV\ChIJ4Qa_rxI4zDERb7Q-H7PIiuA_9.jpg
[1.000 0.572 0.977 0.996 0.002 1.000]
187
D:\FYP\KL_GSV\ChIJq56QqhI4zDERK-E-yCMmrI0_9.jpg
[0.997 0.316 0.158 0.958 0.002 0.998]
188
D:\FYP\KL_GSV\ChIJOb1voxI4zDERSr0z_tQSb_Y_9.jpg
[0.902 0.292 0.181 0.979 0.159 0.892]
189
D:\FYP\KL_GSV\ChIJM2TTuRI4zDERhi90K_PzEoI_9.jpg
[1.000 0.877 0.983 1.000 0.000 1.000]
190
D:\FYP\KL_GSV\ChIJZYaMpxI4zDERFYSHuftt1LY_9.jpg
[0.322 0.433 0.107 0.648 0.058 0.894]
191
D:\FYP\KL_GSV\ChIJOb1voxI4zDERS70z_tQSb_Y_9.jpg
[0.910 0.873 0.715 0.987 0.001 0.947]
192
D:\FYP\KL_GSV\ChIJD8_dWQ04zDERdCyHql9rrMY_9.jpg
[0.998 0.027 0.203 1.000 0.010 0.989]
193
D:\FYP\KL_GSV\ChIJG8BNyhI4zDERV2CfCukGhEA_9.jpg
[0.994 0.998 0.325 0.992 0.001 0.997]
194
D:\FYP\KL_

D:\FYP\KL_GSV\ChIJy2-POww4zDER1-vnL80fbfE_18.jpg
[0.879 0.636 0.134 0.996 0.019 0.901]
364
D:\FYP\KL_GSV\ChIJs5Fmagw4zDERbGZVaLGjkKQ_18.jpg
[0.973 0.464 0.980 0.987 0.385 0.738]
365
D:\FYP\KL_GSV\ChIJy2-POww4zDER1uvnL80fbfE_18.jpg
[0.041 0.434 0.996 0.966 0.620 0.088]
366
D:\FYP\KL_GSV\ChIJ89gmTgw4zDERItEfNOjrmRI_18.jpg
[0.982 0.663 0.023 0.987 0.364 0.890]
367
D:\FYP\KL_GSV\ChIJCencSww4zDERhksiehDKlxs_18.jpg
[0.979 0.663 0.037 0.987 0.236 0.990]
368
D:\FYP\KL_GSV\ChIJI4WUJQw4zDER5lockJQ6kTo_18.jpg
[0.940 0.013 0.059 0.433 0.209 0.983]
369
D:\FYP\KL_GSV\ChIJHQv9PAw4zDERuuc2osZEk9A_18.jpg
[0.776 0.614 0.735 0.986 0.010 0.447]
370
D:\FYP\KL_GSV\ChIJHQv9PAw4zDERjGTx5q-dHmU_18.jpg
[0.995 0.027 0.059 0.967 0.577 0.746]
371
D:\FYP\KL_GSV\ChIJ5Sj3Qgw4zDER4vdgShCf1WA_18.jpg
[0.897 0.046 0.020 0.313 0.046 0.106]
372
D:\FYP\KL_GSV\ChIJkQo-OAw4zDERHv0dyivLxT4_18.jpg
[0.996 0.907 0.444 0.782 0.149 0.282]
373
D:\FYP\KL_GSV\ChIJGxV0Pgw4zDERDgRdasCKLvA_18.jpg
[0.973 0.116 0.633 0.984 0.156 0.835]
374

D:\FYP\KL_GSV\ChIJ8ergAWs4zDERvievaqhWXzA_26.jpg
[0.830 0.030 0.145 0.638 0.233 0.995]
544
D:\FYP\KL_GSV\ChIJzdxlWmo4zDERdrMs8DglMgw_26.jpg
[0.992 0.588 0.062 1.000 0.045 1.000]
545
D:\FYP\KL_GSV\ChIJ4-iv_Go4zDER5tZnUOVDMPE_26.jpg
[0.998 0.644 0.492 0.975 0.010 0.054]
546
D:\FYP\KL_GSV\ChIJFUxtpms4zDERodGdst0z_lY_26.jpg
[1.000 0.031 0.021 0.968 0.338 1.000]
547
D:\FYP\KL_GSV\ChIJ1-gZsms4zDER1AKLvfKsSeA_26.jpg
[0.999 0.490 0.001 1.000 0.535 0.974]
548
D:\FYP\KL_GSV\ChIJ91GGBWo4zDER2lgAew4jmDM_26.jpg
[0.891 0.844 0.085 0.993 0.005 1.000]
549
D:\FYP\KL_GSV\ChIJF1pyiWo4zDERPn2whnccV4k_26.jpg
[0.776 0.942 0.365 0.675 0.012 0.092]
550
D:\FYP\KL_GSV\ChIJK4OcXGo4zDERHWucnIkpymc_26.jpg
[0.919 0.863 0.451 1.000 0.018 0.543]
551
D:\FYP\KL_GSV\ChIJZfhhTmo4zDERQioOU5WZgPQ_26.jpg
[0.999 1.000 0.007 0.973 0.240 0.893]
552
D:\FYP\KL_GSV\ChIJgaAeAGo4zDERMmc5VoEfA1M_26.jpg
[0.997 0.364 0.168 0.937 0.008 0.376]
553
D:\FYP\KL_GSV\ChIJd-GyAGo4zDERHr3evQ_xa8g_26.jpg
[0.111 0.999 0.629 0.983 0.134 0.119]
554

D:\FYP\KL_GSV\ChIJ63khcz84zDER8vxOuES8DNY_32.jpg
[0.186 0.994 0.981 0.945 0.000 0.997]
724
D:\FYP\KL_GSV\ChIJeewQ3zs4zDEREkrL88D-XW4_32.jpg
[0.912 0.550 0.003 0.308 0.341 0.813]
725
D:\FYP\KL_GSV\ChIJFwHdojk4zDERSmJnlOAjdfw_32.jpg
[0.930 0.896 0.958 0.822 0.016 0.429]
726
D:\FYP\KL_GSV\ChIJtfQ5kjc4zDERQCiKtGF1vMo_32.jpg
[0.996 0.166 0.337 0.423 0.829 0.987]
727
D:\FYP\KL_GSV\ChIJf7_WZDk4zDERohcjoA-bCYw_32.jpg
[0.898 0.980 0.858 0.909 0.097 0.808]
728
D:\FYP\KL_GSV\ChIJlaZN1Dk4zDERJG-ySw2fchk_32.jpg
[0.943 0.207 0.923 0.987 0.080 0.998]
729
D:\FYP\KL_GSV\ChIJofCqlDk4zDER1gkewIErdTA_32.jpg
[0.972 0.017 0.163 0.981 0.287 1.000]
730
D:\FYP\KL_GSV\ChIJLYEjhzk4zDERo_J0D2JVL3I_32.jpg
[0.995 0.007 0.358 0.971 0.590 0.999]
731
D:\FYP\KL_GSV\ChIJiXrLZjk4zDERlhLJfuW3ai0_32.jpg
[0.073 0.463 0.005 0.998 0.010 1.000]
732
D:\FYP\KL_GSV\ChIJn83Wwjk4zDEROmpE3BsJUp8_32.jpg
[0.999 0.000 0.050 0.905 0.717 1.000]
733
D:\FYP\KL_GSV\ChIJJ4swmTk4zDERQG7-sBO5VOg_32.jpg
[0.958 0.986 0.837 0.742 0.065 0.461]
734

...

D:\FYP\KL_GSV\ChIJhalzUi1IzDERv6zBtW-sWaY_192.jpg
[0.918 1.000 0.946 1.000 0.001 0.016]
3555
D:\FYP\KL_GSV\ChIJNatNSyxIzDERkqYeboYR4R8_192.jpg
[0.998 0.967 0.794 1.000 0.003 1.000]
3556
D:\FYP\KL_GSV\ChIJQU7G1S1IzDERbvz_0E8mZNs_192.jpg
[0.111 0.184 1.000 0.971 0.000 1.000]
3557
D:\FYP\KL_GSV\ChIJm2v7JC1IzDER4UQneTNIyUE_192.jpg
[0.688 0.375 0.108 0.997 0.014 1.000]
3558
D:\FYP\KL_GSV\ChIJMcYddC1IzDERoeTSFjU0rl8_192.jpg
[0.998 0.129 0.769 0.999 0.014 1.000]
3559
D:\FYP\KL_GSV\ChIJM136QS1IzDERcFmJfCDFTnY_192.jpg
[0.988 1.000 0.945 1.000 0.009 0.207]
3560
D:\FYP\KL_GSV\ChIJe9bBVc1JzDERnx0I2WKXeTA_192.jpg
[0.999 0.007 0.010 0.995 0.058 1.000]
3561
D:\FYP\KL_GSV\ChIJ8-DeXC1IzDERpLV12p9hRMU_192.jpg
[1.000 0.101 0.576 1.000 0.039 0.990]
3562
D:\FYP\KL_GSV\ChIJMysxty1IzDERp2dTc20vQ7A_192.jpg
[1.000 0.998 0.951 0.998 0.003 1.000]
3563
D:\FYP\KL_GSV\ChIJ9bo-ri1IzDER11Wrzolky6o_192.jpg
[0.925 0.999 0.998 0.999 0.000 0.998]
3564
D:\FYP\KL_GSV\ChIJN0uYKSg2zDERYfvBdXXCdkE_192.jpg
[0.999 0.872 0.036 1

D:\FYP\KL_GSV\ChIJRbpAvyBIzDERZuP2muxAAzU_201.jpg
[0.641 0.987 0.978 1.000 0.018 0.892]
3731
D:\FYP\KL_GSV\ChIJ3fNnxd43zDERnkA2m_9Z9yU_201.jpg
[0.993 0.002 0.053 0.989 0.075 0.897]
3732
D:\FYP\KL_GSV\ChIJUaWjOt83zDER3joewhCGo7o_201.jpg
[0.997 0.970 0.013 0.999 0.021 0.894]
3733
D:\FYP\KL_GSV\ChIJ80nbYjJIzDERVomS4UxPBeA_201.jpg
[0.988 0.338 0.993 0.998 0.000 1.000]
3734
D:\FYP\KL_GSV\ChIJZ24rhjJIzDERBm9Ob1Np250_201.jpg
[0.987 0.998 0.994 0.992 0.005 0.519]
3735
D:\FYP\KL_GSV\ChIJ3U24By5IzDER8zz2NZokDvA_201.jpg
[0.935 0.014 0.156 1.000 0.001 1.000]
3736
D:\FYP\KL_GSV\ChIJU_yZfzJIzDERnnWOXB4r5os_201.jpg
[0.313 0.300 0.966 0.815 0.035 0.606]
3737
D:\FYP\KL_GSV\ChIJiZfxjds3zDER4XaDkQ_X2ng_202.jpg
[0.131 0.998 0.990 0.927 0.000 0.102]
3738
D:\FYP\KL_GSV\ChIJFReP7Ns3zDERIbaqahzkmh4_202.jpg
[0.985 0.310 0.052 0.471 0.001 0.592]
3739
D:\FYP\KL_GSV\ChIJ15gcC9w3zDERIWB3VlPE6vM_202.jpg
[0.992 0.001 0.606 0.999 0.257 1.000]
3740
D:\FYP\KL_GSV\ChIJheavnNs3zDERfj_YxBrOKSQ_202.jpg
[0.926 0.044 0.000 0

D:\FYP\KL_GSV\ChIJmXooKb43zDER6B12cI5PMDg_98.jpg
[0.971 0.277 0.001 0.991 0.643 0.633]
3907
D:\FYP\KL_GSV\ChIJ28tcLb43zDERNq-9oj9oIp4_98.jpg
[0.998 0.866 0.009 0.963 0.008 0.904]
3908
D:\FYP\KL_GSV\ChIJ9RfxS743zDER6D7RjGZlqGE_98.jpg
[0.989 0.300 0.005 0.997 0.131 0.356]
3909
D:\FYP\KL_GSV\ChIJQ5oTRr43zDERCuHBsamZRPI_98.jpg
[0.631 0.051 0.013 0.467 0.309 0.140]
3910
D:\FYP\KL_GSV\ChIJVe5OSr43zDERtjss9DJnV28_98.jpg
[0.117 0.017 0.223 0.979 0.119 0.144]
3911
D:\FYP\KL_GSV\ChIJHbkZAr43zDERm4ZvZu4jVlA_98.jpg
[1.000 0.033 0.031 0.990 0.113 0.950]
3912
D:\FYP\KL_GSV\ChIJXS7Ix903zDERJt9ScoC6Wq8_98.jpg
[1.000 0.884 0.002 0.999 0.003 0.999]
3913
D:\FYP\KL_GSV\ChIJXS4yvd03zDERXBTjrAoJMdA_98.jpg
[0.785 0.303 0.769 0.991 0.882 0.149]
3914
D:\FYP\KL_GSV\ChIJczjnst03zDERrXy9Bi9ayPA_98.jpg
[0.999 0.275 0.062 0.833 0.776 0.017]
3915
D:\FYP\KL_GSV\ChIJO1ePjec3zDERtlmDC4CWNhk_98.jpg
[0.986 0.940 0.425 0.996 0.012 0.686]
3916
D:\FYP\KL_GSV\ChIJuxwOY903zDERyqR2HE2qvBQ_98.jpg
[0.981 0.136 0.212 0.814 0.114 

D:\FYP\KL_GSV\ChIJVZFaqTxIzDERggEEahEw5SI_219.jpg
[0.987 0.845 0.999 0.898 0.055 0.992]
4083
D:\FYP\KL_GSV\ChIJ3xa1RCVIzDERXuk3ej3vxjY_219.jpg
[0.142 0.745 0.977 0.940 0.026 0.682]
4084
D:\FYP\KL_GSV\ChIJJW8zzCRIzDERqnFcVyf57Fg_219.jpg
[0.773 1.000 0.990 0.990 0.019 0.983]
4085
D:\FYP\KL_GSV\ChIJeVkb9h44zDERUJt7_l3ySws_220.jpg
[0.818 0.621 0.287 0.605 0.005 0.986]
4086
D:\FYP\KL_GSV\ChIJ8dJl6R44zDER4B6GmUKjAho_220.jpg
[0.482 0.523 0.219 0.280 0.407 0.967]
4087
D:\FYP\KL_GSV\ChIJ-25RFh44zDERrqeXTe4LosQ_220.jpg
[0.993 0.583 0.112 0.916 0.185 0.966]
4088
D:\FYP\KL_GSV\ChIJD9RTQx44zDEREqNIMNV1Z5Q_220.jpg
[0.971 0.969 0.816 0.482 0.066 0.998]
4089
D:\FYP\KL_GSV\ChIJF_-EYh44zDERvRU_A-WsIjs_220.jpg
[0.318 0.980 0.999 0.092 0.001 0.071]
4090
D:\FYP\KL_GSV\ChIJk2uQlh44zDER0ELSkDRZWHw_220.jpg
[0.998 0.935 0.152 0.974 0.041 0.943]
4091
D:\FYP\KL_GSV\ChIJUS-hZx44zDERNLE0ygKxUNQ_220.jpg
[0.992 0.996 0.031 0.988 0.011 0.966]
4092
D:\FYP\KL_GSV\ChIJB4sz7h44zDERVBEDsoHdgnQ_220.jpg
[0.980 0.155 0.946 0

[0.142 0.086 0.015 0.448 0.600 0.999]
4259
D:\FYP\KL_GSV\ChIJrVmPQho4zDERO7xaQ8QpPbo_65.jpg
[0.829 0.050 0.978 0.032 0.007 0.100]
4260
D:\FYP\KL_GSV\ChIJcRiU4ho4zDERgHTUISRKxOA_65.jpg
[0.600 0.972 0.879 1.000 0.000 1.000]
4261
D:\FYP\KL_GSV\ChIJJRhvCxA4zDERHZrPjPeNFa8_65.jpg
[0.701 0.896 0.943 0.999 0.000 0.932]
4262
D:\FYP\KL_GSV\ChIJFcdtERo4zDERjMhD1irmFOg_65.jpg
[0.960 0.610 0.861 0.657 0.002 0.141]
4263
D:\FYP\KL_GSV\ChIJ_7FTUxc4zDERhto9vlCKZJ8_65.jpg
[0.945 0.975 0.897 0.990 0.480 0.999]
4264
D:\FYP\KL_GSV\ChIJ1XCGTgU4zDERxa8J_m9EtWA_65.jpg
[0.971 0.983 0.261 0.964 0.099 0.995]
4265
D:\FYP\KL_GSV\ChIJz7L8rRo4zDERSI-mSjy-Ftc_65.jpg
[0.936 0.996 0.214 0.996 0.000 0.668]
4266
D:\FYP\KL_GSV\ChIJ24d1ohs4zDERaqjin5tcq78_65.jpg
[0.923 0.998 0.843 0.009 0.000 0.974]
4267
D:\FYP\KL_GSV\ChIJ_ToYYxA4zDER5pEnziNjc7k_65.jpg
[0.901 0.987 0.209 0.998 0.005 0.999]
4268
D:\FYP\KL_GSV\ChIJOQ0RrBA4zDERtFgQiG0dQyQ_65.jpg
[0.898 0.787 0.105 0.985 0.037 0.999]
4269
D:\FYP\KL_GSV\ChIJj9wepBA4zDERuKE7M3I

In [67]:
df = df.reset_index(drop=True)

#### Append predicted result to df

In [68]:
# append predicted result to df

labels = pd.DataFrame(predicted_scores)
labels.columns = ["beautiful","boring","depressing","lively","safety","wealthy"]
labelled_df = df.join(labels) 
labelled_df
labelled_df.to_csv(r"D:\FYP\Visualisation\labelled_KL_latest_cleaned_latest.csv")

In [69]:
labelled_df

,Unnamed: 0,filename,latitude,longitude,neighbourhoodID,panoID,beautiful,boring,depressing,lively,safety,wealthy
0,0,ChIJYbEOt2Y4zDER1rRiQCmnKpk_0.jpg,3.224395,101.725032,0,ChIJYbEOt2Y4zDER1rRiQCmnKpk,0.957515,0.086774,0.273894,0.949782,0.479761,0.791962
1,1,ChIJf7L63GY4zDERtn4RRBJ5j8g_0.jpg,3.223890,101.726111,0,ChIJf7L63GY4zDERtn4RRBJ5j8g,0.824439,0.092749,0.898962,0.968768,0.007706,0.260710
2,2,ChIJWZXLyGY4zDER8HrwgxBiXIM_0.jpg,3.224468,101.725888,0,ChIJWZXLyGY4zDER8HrwgxBiXIM,0.992937,0.004568,0.005562,0.995972,0.002129,0.996372
3,3,ChIJBRqW62Y4zDER5hk-VslDi6o_0.jpg,3.223750,101.725436,0,ChIJBRqW62Y4zDER5hk-VslDi6o,0.961063,0.643230,0.784337,0.998392,0.212676,0.613903
4,4,ChIJVVCnx2Y4zDERuMiq6zyzIXs_0.jpg,3.223907,101.725502,0,ChIJVVCnx2Y4zDERuMiq6zyzIXs,0.992860,0.525083,0.952074,0.972225,0.009295,0.992308
5,5,ChIJ4XieRGE4zDERdtE5f_yE81g_0.jpg,3.225625,101.724582,0,ChIJ4XieRGE4zDERdtE5f_yE81g,0.300873,0.937709,0.643009,0.495441,0.005246,0.990669
6,6,ChIJh7QNS2E4zDER8uQZQewf8Vo_0.jpg,3.225141,101.725059,0,ChIJh7QNS2E4zDER8uQZQewf8Vo,0.922562,0.154539,0.051329,0.997188,0.006498,0.999066
7,7,ChIJB6-MrWY4zDERQE4s69HmJfM_0.jpg,3.224915,101.724679,0,ChIJB6-MrWY4zDERQE4s69HmJfM,0.648981,0.735116,0.949282,0.999896,0.220985,0.954326
8,8,ChIJQ1hPOGE4zDERdkZjREI-6no_0.jpg,3.226873,101.724550,0,ChIJQ1hPOGE4zDERdkZjREI-6no,0.690936,0.950067,0.014109,0.998876,0.001584,0.998305
9,9,ChIJbQmI-WM4zDERBWb7oyT8gVs_0.jpg,3.225418,101.723208,0,ChIJbQmI-WM4zDERBWb7oyT8gVs,0.994389,0.958028,0.064764,0.818915,0.005233,0.935344
